In [18]:
import youtube_dl
import os
import sys
import shutil
from moviepy.editor import VideoFileClip, concatenate_videoclips
import numpy as np

# from VideoSync.alignment_by_row_channels import *

from cvcalib.audiosync import offset

curr_dir = "/home/sean/Documents/experiments/stage-mix-generator/"
download_dir = os.path.join(curr_dir, 'temp')

In [2]:
# get videos via youtube api
lyrics_video_url = "https://www.youtube.com/watch?v=mIXR-7u2Kas"
stage_video_urls = [
    "https://www.youtube.com/watch?v=kda_53A0ZpI",
    "https://www.youtube.com/watch?v=zzq8uI1BnJk",
    "https://www.youtube.com/watch?v=Q65bS5EyX5I",
    "https://www.youtube.com/watch?v=QsV25Q6VldA"
]

In [4]:
# download videos

ydl_opts = {
    'format': 'mp4',
    'outtmpl': os.path.join(download_dir, '%(id)s.%(ext)s')
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([lyrics_video_url])
    meta = ydl.extract_info(
        lyrics_video_url, download=False)
    
    shutil.move(os.path.join(download_dir, f"{meta['id']}.mp4"), os.path.join(download_dir, "lyrics_video.mp4"))
    
    for i, url in enumerate(stage_video_urls):
        ydl.download([url])
        meta = ydl.extract_info(
            url, download=False)
        
        shutil.move(os.path.join(download_dir, f"{meta['id']}.mp4"), os.path.join(download_dir, f"stage_video{i}.mp4"))

[youtube] mIXR-7u2Kas: Downloading webpage
[youtube] mIXR-7u2Kas: Downloading video info webpage
[download] Destination: /home/sean/Documents/experiments/stage-mix-generator/temp/mIXR-7u2Kas.mp4
[download] 100% of 8.88MiB in 00:0079MiB/s ETA 00:000
[youtube] mIXR-7u2Kas: Downloading webpage
[youtube] mIXR-7u2Kas: Downloading video info webpage
[youtube] kda_53A0ZpI: Downloading webpage
[youtube] kda_53A0ZpI: Downloading video info webpage
[download] Destination: /home/sean/Documents/experiments/stage-mix-generator/temp/kda_53A0ZpI.mp4
[download] 100% of 25.29MiB in 00:0164MiB/s ETA 00:003
[youtube] kda_53A0ZpI: Downloading webpage
[youtube] kda_53A0ZpI: Downloading video info webpage
[youtube] zzq8uI1BnJk: Downloading webpage
[youtube] zzq8uI1BnJk: Downloading video info webpage
[download] Destination: /home/sean/Documents/experiments/stage-mix-generator/temp/zzq8uI1BnJk.mp4
[download] 100% of 21.99MiB in 00:0136MiB/s ETA 00:008
[youtube] zzq8uI1BnJk: Downloading webpage
[youtube] zzq8

In [15]:
lyrics_video_path = os.path.join(download_dir, 'lyrics_video.mp4')
stage_video_paths = [ os.path.join(download_dir, f'stage_video{i}.mp4') for i in range(len(stage_video_urls)) ]

# find timestamp of start of song in stage videos
# t = align('lyrics_video.mp4', 'stage_video1.mp4', download_dir+'/')

offset.find_time_offset(['lyrics_video.mp4', 'stage_video0.mp4'], download_dir+'/', [0, 0])

((0, 32.8446), 29.97)

In [16]:
stage_song_starts = []
for i in range(len(stage_video_urls)):
    output = offset.find_time_offset(['lyrics_video.mp4', f'stage_video{i}.mp4'], download_dir+'/', [0, 0])
    stage_song_starts.append(output[0][1])

In [17]:
stage_song_starts

[32.8446, 0.1161, 7.0937, 0]

In [7]:
# get subclips of the song in the stage videos

# get length of lyrics video
song = VideoFileClip(lyrics_video_path)

stage_videos = []
for i, path in enumerate(stage_video_paths):
    stage_videos.append(VideoFileClip(path).subclip(stage_song_starts[i]))

In [12]:
# every 5 seconds during the song, splice clips from different performances

clips = []
stage_video_idx = 0
for step in np.arange(0, song.duration, 5):
    clips.append(stage_videos[stage_video_idx].subclip(step, step+5))
    
    stage_video_idx = (stage_video_idx + 1) % len(stage_videos)
    
final_clip = concatenate_videoclips(clips)
final_clip.write_videofile("stage_mix.mp4")

                                                                      
chunk:  99%|█████████▊| 4783/4852 [02:50<00:00, 1045.87it/s, now=None]
                                                                      [A
chunk:  99%|█████████▊| 4783/4852 [00:33<00:00, 844.17it/s, now=None]

chunk:   0%|          | 0/4741 [00:00<?, ?it/s, now=None]

chunk:   1%|          | 32/4741 [00:00<00:17, 263.87it/s, now=None]

Moviepy - Building video stage_mix.mp4.
MoviePy - Writing audio in %s




chunk:   2%|▏         | 112/4741 [00:00<00:15, 308.43it/s, now=None]

chunk:   4%|▍         | 207/4741 [00:00<00:11, 386.76it/s, now=None]

chunk:   5%|▌         | 251/4741 [00:00<00:11, 387.96it/s, now=None]

chunk:   7%|▋         | 332/4741 [00:00<00:10, 424.43it/s, now=None]

chunk:   9%|▉         | 433/4741 [00:00<00:08, 513.52it/s, now=None]

chunk:  11%|█         | 521/4741 [00:00<00:07, 586.68it/s, now=None]

chunk:  13%|█▎        | 601/4741 [00:00<00:06, 637.07it/s, now=None]

chunk:  14%|█▍        | 679/4741 [00:01<00:06, 671.11it/s, now=None]

chunk:  16%|█▋        | 773/4741 [00:01<00:05, 716.31it/s, now=None]

chunk:  18%|█▊        | 875/4741 [00:01<00:04, 784.91it/s, now=None]

chunk:  20%|██        | 968/4741 [00:01<00:04, 821.20it/s, now=None]

chunk:  22%|██▏       | 1056/4741 [00:01<00:04, 837.64it/s, now=None]

chunk:  24%|██▍       | 1145/4741 [00:01<00:04, 851.01it/s, now=None]

chunk:  26%|██▌       | 1236/4741 [00:01<00:04, 864.62it/s, now=None]

chunk:  28%|██▊

MoviePy - Done.
Moviepy - Writing video stage_mix.mp4





t:   1%|          | 79/6444 [00:00<00:16, 392.65it/s, now=None]

t:   2%|▏         | 108/6444 [00:00<00:17, 353.51it/s, now=None]

t:   2%|▏         | 136/6444 [00:00<00:19, 326.97it/s, now=None]

t:   2%|▏         | 161/6444 [00:00<00:27, 232.28it/s, now=None]

t:   3%|▎         | 188/6444 [00:00<00:26, 237.35it/s, now=None]

t:   3%|▎         | 211/6444 [00:00<00:27, 223.28it/s, now=None]

t:   4%|▎         | 233/6444 [00:00<00:29, 210.92it/s, now=None]

t:   4%|▍         | 254/6444 [00:01<00:30, 202.58it/s, now=None]

t:   4%|▍         | 274/6444 [00:01<00:31, 195.77it/s, now=None]

t:   5%|▍         | 299/6444 [00:01<00:29, 207.32it/s, now=None]

t:   5%|▍         | 320/6444 [00:01<00:35, 172.89it/s, now=None]

t:   5%|▌         | 339/6444 [00:01<00:37, 164.22it/s, now=None]

t:   6%|▌         | 359/6444 [00:01<00:35, 172.76it/s, now=None]

t:   6%|▌         | 382/6444 [00:01<00:32, 186.30it/s, now=None]

t:   6%|▋         | 403/6444 [00:01<00:31, 191.03it/s, now=None]

t:   7%|▋

t:  38%|███▊      | 2443/6444 [00:13<00:26, 148.79it/s, now=None]

t:  38%|███▊      | 2461/6444 [00:13<00:25, 153.94it/s, now=None]

t:  39%|███▊      | 2483/6444 [00:13<00:23, 168.22it/s, now=None]

t:  39%|███▉      | 2501/6444 [00:13<00:23, 167.00it/s, now=None]

t:  39%|███▉      | 2521/6444 [00:13<00:22, 175.48it/s, now=None]

t:  39%|███▉      | 2544/6444 [00:14<00:20, 187.88it/s, now=None]

t:  40%|███▉      | 2564/6444 [00:14<00:24, 159.82it/s, now=None]

t:  40%|████      | 2582/6444 [00:14<00:23, 162.43it/s, now=None]

t:  40%|████      | 2601/6444 [00:14<00:22, 169.00it/s, now=None]

t:  41%|████      | 2623/6444 [00:14<00:21, 180.75it/s, now=None]

t:  41%|████      | 2642/6444 [00:14<00:21, 180.56it/s, now=None]

t:  41%|████▏     | 2661/6444 [00:14<00:21, 179.15it/s, now=None]

t:  42%|████▏     | 2680/6444 [00:14<00:20, 182.00it/s, now=None]

t:  42%|████▏     | 2700/6444 [00:15<00:26, 143.62it/s, now=None]

t:  42%|████▏     | 2732/6444 [00:15<00:21, 171.64it/s, now=No

t:  74%|███████▍  | 4780/6444 [00:26<00:09, 170.91it/s, now=None]

t:  74%|███████▍  | 4798/6444 [00:27<00:12, 126.63it/s, now=None]

t:  75%|███████▌  | 4834/6444 [00:27<00:10, 155.98it/s, now=None]

t:  75%|███████▌  | 4855/6444 [00:27<00:09, 163.66it/s, now=None]

t:  76%|███████▌  | 4876/6444 [00:27<00:09, 171.73it/s, now=None]

t:  76%|███████▌  | 4897/6444 [00:27<00:08, 180.31it/s, now=None]

t:  76%|███████▋  | 4917/6444 [00:27<00:08, 178.11it/s, now=None]

t:  77%|███████▋  | 4937/6444 [00:27<00:08, 172.32it/s, now=None]

t:  77%|███████▋  | 4956/6444 [00:27<00:10, 148.63it/s, now=None]

t:  77%|███████▋  | 4980/6444 [00:27<00:08, 167.60it/s, now=None]

t:  78%|███████▊  | 5003/6444 [00:28<00:07, 182.10it/s, now=None]

t:  78%|███████▊  | 5026/6444 [00:28<00:07, 193.86it/s, now=None]

t:  78%|███████▊  | 5048/6444 [00:28<00:06, 201.00it/s, now=None]

t:  79%|███████▊  | 5070/6444 [00:28<00:07, 193.33it/s, now=None]

t:  79%|███████▉  | 5091/6444 [00:28<00:07, 190.75it/s, now=No

Moviepy - Done !
Moviepy - video ready stage_mix.mp4
